<a href="https://colab.research.google.com/github/tsitsvero/rotor-gp/blob/main/pack/train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [1]:
!pip install git+https://github.com/lab-cosmo/librascal gpytorch pytorch-lightning wandb git+https://github.com/chem-gp/fande xtb nbstripout
! nbstripout --install --global

  Cloning https://github.com/lab-cosmo/librascal to /tmp/pip-req-build-eo9ugbmz
  Running command git clone --filter=blob:none --quiet https://github.com/lab-cosmo/librascal /tmp/pip-req-build-eo9ugbmz
  Resolved https://github.com/lab-cosmo/librascal to commit 6c55e99720f9a181f1efc5c7cd4976c87809d79b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/chem-gp/fande to /tmp/pip-req-build-b23zn9dh
  Running command git clone --filter=blob:none --quiet https://github.com/chem-gp/fande /tmp/pip-req-build-b23zn9dh
  Resolved https://github.com/chem-gp/fande to commit 353b1733424bfdd15a117ac41dd5f909ed5ce40c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.6 MB/s eta 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!wget https://raw.githubusercontent.com/tsitsvero/rotor-gp/main/pack/cook_model.py

--2023-12-31 14:21:11--  https://raw.githubusercontent.com/tsitsvero/rotor-gp/main/pack/cook_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9070 (8.9K) [text/plain]
Saving to: ‘cook_model.py’

cook_model.py       100%[===================>]   8.86K  --.-KB/s    in 0s      

2023-12-31 14:21:11 (80.2 MB/s) - ‘cook_model.py’ saved [9070/9070]



## Cooking up the model

In [8]:
# Prepare the training data:

from ase import io
import os

# DATA_DIR = os.path.expanduser("~/repos/data/")
# # DATA_DIR = "/data1/simulations/datasets/rotors/high_temp_ML_training_data/"
# RESULTS_DIR = os.path.expanduser("~/repos/data/results")

DATA_DIR = os.path.expanduser("/content/drive/MyDrive/data/")
# FANDE_DIR = os.path.expanduser("~/")
RESULTS_DIR = os.path.expanduser("/content/drive/MyDrive/data/results/")
os.makedirs(RESULTS_DIR, exist_ok=True)

PREPARE_DATA = False
if PREPARE_DATA:
        traj_295 = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index=":")
        # traj_355 = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")
        traj_295_2000K = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
        # traj_355_2000K = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
        # traj_295_2000K_forced = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
        # traj_355_2000K_forced = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")

        # trajectory_energy = traj_295[0:5000] + traj_355[0:5000] + traj_295_2000K[0:5000] + traj_355_2000K[0:5000] + traj_295_2000K_forced[0:5000] + traj_355_2000K_forced[0:5000]

        train_energy = traj_295[0:5000:10] +  traj_295_2000K[0:5000:10]
        train_forces = traj_295_2000K[0:5000:5]
        validation_energy = traj_295[1000:1010]
        validation_forces = traj_295[1000:1010]

        io.write(DATA_DIR+"/train_energy.traj", train_energy)
        io.write(DATA_DIR+"/train_forces.traj", train_forces)
        io.write(DATA_DIR+"/validation_energy.traj", validation_energy)
        io.write(DATA_DIR+"/validation_forces.traj", validation_forces)


In [13]:
ENERGY_MODEL = 'variational_inducing_points' #'variational_inducing_points', 'exact'
ENERGY_NUM_INDUCING_POINTS = 10
ENERGY_LR = 0.1
ENERGY_NUM_STEPS = 5

FORCES_MODEL = 'variational_inducing_points' #'variational_inducing_points', 'exact'
FORCES_NUM_INDUCING_POINTS = 10
NUM_FORCE_SAMPLES = 10
FORCES_LR = 0.1
FORCES_NUM_STEPS = 5

PREDICTOR_NAME = 'test.pth'
SUBSAMPLE = 200 # subsample data to reduce time durings tests

! python cook_model.py \
--data_dir $DATA_DIR \
--results_dir $RESULTS_DIR \
--energy_model $ENERGY_MODEL \
--energy_num_inducing_points $ENERGY_NUM_INDUCING_POINTS \
--energy_lr $ENERGY_LR \
--energy_num_steps $ENERGY_NUM_STEPS \
--forces_model $FORCES_MODEL \
--forces_num_inducing_points $FORCES_NUM_INDUCING_POINTS \
--num_force_samples $NUM_FORCE_SAMPLES \
--forces_lr $FORCES_LR \
--forces_num_steps $FORCES_NUM_STEPS \
--predictor_name $PREDICTOR_NAME \
--subsample $SUBSAMPLE


DATA_DIR /content/drive/MyDrive/data/
RESULTS_DIR /content/drive/MyDrive/data/results/
ENERGY_MODEL variational_inducing_points
ENERGY_NUM_INDUCING_POINTS 10
ENERGY_LR 0.1
ENERGY_NUM_STEPS 5
FORCES_MODEL variational_inducing_points
FORCES_NUM_INDUCING_POINTS 10
NUM_FORCE_SAMPLES 10
FORCES_LR 0.1
FORCES_NUM_STEPS 5
PREDICTOR_NAME test.pth
5 5
fande.models module imported...
Icecream logger is not available
Total number of found groups: 14
Checking if all atoms are covered:  True
Preparing dataloaders...
Total length of traj is 5
Total number of batches 5
Calculating invariants on trajectory with librascal...
100% 5/5 [00:00<00:00, 17.94it/s]
invariants for energy fitting calculated
100% 5/5 [00:08<00:00,  1.75s/it]
invariants for forces fitting calculated
torch.Size([5, 1200])
torch.Size([5])
Dataloader for group 0 created
Number of samples in dataloader: 10
Dataloader for group 1 created
Number of samples in dataloader: 10
Dataloader for group 2 created
Number of samples in dataloader:

## Testing models

In [14]:
from ase import io
test_traj = io.read(DATA_DIR + "/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="1000:1010")

In [15]:
import sys
import torch
FANDE_DIR = os.path.expanduser("~/repos/")
sys.path.append(FANDE_DIR + "fande/")

from fande.predict import FandePredictor
from fande.ase import FandeCalc
# load the predictor:
# predictor_loaded = torch.load(RESULTS_DIR + "/fande_predictor.pth")
predictor_loaded = torch.load(RESULTS_DIR + "/test.pth")
fande_calc_loaded = FandeCalc(predictor_loaded)
device = torch.device('cpu')
fande_calc_loaded.predictor.move_models_to_device(device)

No module named 'skmatter'


In [16]:
forces_true = []
forces_predicted = []

energy_true = []
energy_predicted = []

for atoms in test_traj:
    forces_true.append(atoms.get_forces())
    energy_true.append(atoms.get_potential_energy())
    atoms.set_calculator(fande_calc_loaded)
    forces_predicted.append(atoms.get_forces())
    energy_predicted.append(atoms.get_potential_energy())

INFO:fande:Setting context for descriptors calculation to production
INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1640.7017707824707
Predicting forces...
Energy model summary: 
Time invariants:  0.1819133758544922
Time prediction:  6.583929061889648
Time moving on device:  0.9796619415283203
Time total:  15.606880187988281


INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1565.4616355895996
Predicting forces...
Energy model summary: 
Time invariants:  0.16617774963378906
Time prediction:  13.920068740844727
Time moving on device:  1.1463165283203125
Time total:  23.116111755371094


INFO:fande:Setting context for descriptors calculation to production


...

Time for invariants (call from forces):  1565.1326179504395
Predicting forces...
Energy model summary: 
Time invariants:  0.17023086547851562
Time prediction:  13.309240341186523
Time moving on device:  1.1134147644042969
Time total:  22.389888763427734


INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1562.9172325134277
Predicting forces...
Energy model summary: 
Time invariants:  0.16760826110839844
Time prediction:  6.44230842590332
Time moving on device:  8.201837539672852
Time total:  24.730920791625977


INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1566.2424564361572
Predicting forces...
Energy model summary: 
Time invariants:  0.21719932556152344
Time prediction:  9.682893753051758
Time moving on device:  0.9925365447998047
Time total:  19.509077072143555
Time for invariants (call from forces):  1569.739580154419
Predicting forces...
Energy model summary: 
Time invariants:  0.17404556274414062
Time prediction:  13.674736022949219
Time moving on device:  1.5537738800048828
Time total:  23.237228393554688


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

energy_true = np.array(energy_true)
energy_predicted = np.array(energy_predicted)
energy_errors = energy_predicted - energy_true

forces_true = np.array(forces_true)
forces_predicted = np.array(forces_predicted)
forces_errors = forces_predicted - forces_true

atomic_groups = predictor_loaded.fdm.atomic_groups



for ag in atomic_groups:

    print("Atomic group", ag)
    print("F_x")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 0].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 0].flatten()**2)))
    print("F_y")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 1].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 1].flatten()**2)))
    print("F_z")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 2].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 2].flatten()**2)))
    print("E")
    print("MAE", np.mean(np.abs(energy_errors.flatten())))
    print("RMSE", np.sqrt(np.mean(energy_errors.flatten()**2)))

    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))
    axs[0].set_title(r"$F_x$")
    axs[0].plot(np.array(forces_true)[:, ag, 0].flatten(), label="true")
    axs[0].plot(np.array(forces_predicted)[:, ag, 0].flatten(), label="predicted")
    axs[0].legend()

    axs[1].set_title(r"$F_y$")
    axs[1].plot(np.array(forces_true)[:, ag, 1].flatten(), label="true")
    axs[1].plot(np.array(forces_predicted)[:, ag, 1].flatten(), label="predicted")
    axs[1].legend()

    axs[2].set_title(r"$F_z$")
    axs[2].plot(np.array(forces_true)[:, ag, 2].flatten(), label="true")
    axs[2].plot(np.array(forces_predicted)[:, ag, 2].flatten(), label="predicted")
    axs[2].legend()

    plt.show()

    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))

    axs[0].set_title(r"$F_x$ errors")
    axs[0].hist(forces_errors[:, ag, 0].flatten(), bins=100)

    axs[1].set_title(r"$F_y$ errors")
    axs[1].hist(forces_errors[:, ag, 1].flatten(), bins=100)

    axs[2].set_title(r"$F_z$ errors")
    axs[2].hist(forces_errors[:, ag, 2].flatten(), bins=100)
    plt.show()

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 3))
    axs[0].set_title(r"$E$ errors")
    axs[0].hist(energy_errors.flatten(), bins=100)

    axs[1].set_title(r"$E$")
    axs[1].plot(energy_true, label="true")
    axs[1].plot(energy_predicted, label="predicted")
    axs[1].legend()
    plt.show()
